In [ ]:
import numpy as np
import h5py
from FourierInterp import GetShiftDens, FourierInterpolate
#===========================================================================
class ExcARPES(object):
	"""docstring for ExcARPES"""
	def __init__(self, file_bands, Nk):
		super(ExcARPES, self).__init__()
		self.Nk = Nk

		f = h5py.File(file_bands, "r")
		self.epsk = np.array(f['epsk'])
		f.close()
	#................................................................
	def GetIntensity(self, kpts, ws, exc_en, exc_dens, pexc, iv, eta):
		nk = kpts.shape[0]
		epsv_grid = self.epsk[:,iv-1]
		epsv_max = np.amax(epsv_grid)

		Pexc_shift = GetShiftDens(self.Nk,self.Nk,exc_dens,pexc,normalize=False)
		Pexc_kpts = FourierInterpolate(self.Nk,self.Nk,Pexc_shift,kpts)

		epsv_shift = GetShiftDens(self.Nk,self.Nk,epsv_grid,pexc,normalize=False)
		epsv_kpts = FourierInterpolate(self.Nk,self.Nk,epsv_shift,kpts)

		# spect = gauss(eta, ws[:,None] - epsv_kpts[None,:] - exc_en) * Pexc_kpts[None,:]
		spect = gauss(eta, ws[:,None] - epsv_kpts[None,:] + epsv_max - exc_en) * Pexc_kpts[None,:]
		return spect
#===========================================================================
def gauss(a,x):
	return np.exp(-0.5*(x/a)**2) / np.sqrt(2*np.pi) / a
#==========================================================================

In [ ]:
import numpy as np
#===========================================================================
def FourierInterpolate(nk1,nk2,v,kpts,cplx=False):
    # x = np.reshape(vect[ist,:], [nk1,nk2])
    x = np.reshape(v, [nk1,nk2])
    y = np.fft.fft2(x)

    xr = np.fft.fftfreq(nk1,d=1. / nk1).astype(int)
    yr = np.fft.fftfreq(nk2,d=1. / nk2).astype(int)

    nk = kpts.shape[0]
    if cplx:
        vect_int = np.zeros(nk,dtype=np.complex_)
    else:
        vect_int = np.zeros(nk)
    for ik in range(nk):
        kpt = kpts[ik,:]

        ex1 = np.exp(2j*np.pi*xr*(kpt[0]+0.5))
        ex2 = np.exp(2j*np.pi*yr*(kpt[1]+0.5))

        if cplx:
            vect_int[ik] = p.einsum('i,j,ij', ex1, ex2, y) / (nk1 * nk2)
        else:
            vect_int[ik] = np.real(np.einsum('i,j,ij', ex1, ex2, y)) / (nk1 * nk2)

    return vect_int
#===========================================================================
def GetShiftDens(nk1,nk2,v,pexc,cplx=False,normalize=False):
    x = np.reshape(v, [nk1,nk2])
    y = np.fft.fft2(x)    

    xr = np.fft.fftfreq(nk1,d=1. / nk1).astype(int)
    yr = np.fft.fftfreq(nk2,d=1. / nk2).astype(int)

    ex1 = np.exp(-2j*np.pi*xr*(pexc[0]))
    ex2 = np.exp(-2j*np.pi*yr*(pexc[1]))

    yp = ex1[:,None] * ex2[None,:] * y

    if cplx:
        x = np.fft.ifft2(yp)
    else:
        x = np.real(np.fft.ifft2(yp)) 

    if normalize:
        x = x / (nk1 * nk2)

    return x
#===========================================================================

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import constants as cst
from ReadParameters import ExcitonInput
from ExcitonHamiltonian import Exciton
from ExcitonARPES import ExcARPES
from colormap import cmap_arpes
import kgrid
from honeycomb import DrawBZ
#===========================================================================
def PlotSpectrumCut(ws,spect,klabel):
	fig, ax = plt.subplots()

	xk = np.linspace(0,1,spect.shape[1])
	knode = np.linspace(0,1,len(klabel))
	smax = np.amax(spect)

	mycmap = cmap_arpes()
	ax.imshow(spect, origin="lower", extent=(0,1,ws[0],ws[-1]),\
		vmin=0.0, vmax=smax, cmap=mycmap, interpolation='bilinear')

	for i in range(1,len(knode)-1):
		ax.axvline(x=knode[i],c='k', ls='--', lw=1)

	ax.set_xlim(0,1)
	ax.set_ylim(ws[0],ws[-1])
	ax.set_ylabel(r"$E\,$ (eV)")

	ax.set_xticks(knode)
	ax.set_xticklabels(klabel)
	plt.show()
#===========================================================================
def PlotIntensity(spect,kxr,kyr,nk1,nk2):

	s = np.reshape(spect, [nk1, nk2])
	smax = np.amax(spect)

	fig, ax = plt.subplots()

	mycmap = cmap_arpes()
	ax.imshow(s, origin="lower", extent=(kxr[0], kxr[1], kyr[0], kyr[1]),\
		vmin=0.0, vmax=smax, cmap=mycmap, interpolation='bilinear')

	alat = 3.32/cst.aB
	DrawBZ(alat,ax)

	ax.set_xlabel(r"$k_x$")
	ax.set_ylabel(r"$k_y$")

	plt.show()
#===========================================================================
def main(argv):

	params = ExcitonInput(argv[0])
	#-----------------------------------------------
	#           Define parameters
	#-----------------------------------------------
	exc_par = params.ExcitonParam()
	Nk = exc_par['Nk']
	iv = exc_par['iv']
	ic = exc_par['ic']
	sp = exc_par['sp'] 

	eps_inf = exc_par['eps_inf']
	eps_sub = exc_par['eps_sub']

	nst_Gmm_K = exc_par['nst_Gmm_K']
	nst_Gmm_Kp = exc_par['nst_Gmm_Kp']
	nst_Sig = exc_par['nst_Sig']

	pexcs = [
		np.array([0.0, 0.0]),
		np.array([0.0, 0.0]),
		np.array([0.0, 0.0]),
		np.array([0.16578685, 0.16578685]),
		np.array([-0.16578685, -0.16578685])
	]
	#-----------------------------------------------
	#             Build Exciton
	#-----------------------------------------------
	exc = Exciton(Nk, iv, ic, sp, eps_inf, eps_sub, nst_Gmm_K, nst_Gmm_Kp, nst_Sig)
	exc.BuildHamiltonian(1,sym_rot=True)
	exc.ReadWavefunctions(sym_rot=True)

	arpes_par = params.ARPESParam()
	arpes = ExcARPES(exc_par['file_bands'], Nk)
	#-----------------------------------------------
	#               ARPES
	#-----------------------------------------------
	points = [[-0.5, -0.5], [-1./3., -1./3.], [0., 0.], [1./3., 1./3.], [0.5, 0.5]]
	nseg = 30
	kpts = kgrid.GenKpath(points,nseg)

	eta = arpes_par['eta']
	ws = np.linspace(1.0, 2.2, 100)

	klabel = ["M'", "K'", r"$\Gamma$", "K", "M"]

	for iexc in range(1,exc.nst):
		exc_en = exc.eps[iexc]
		exc_dens = np.abs(exc.excwf[iexc,:])**2
		pexc = pexcs[iexc]
		spect = arpes.GetIntensity(kpts, ws/cst.Ry, exc_en, exc_dens, pexc, iv, eta/cst.Ry)

		PlotSpectrumCut(ws,spect,klabel)

	kxr = arpes_par['kx_range']
	kyr = arpes_par['ky_range']
	nk1, nk2 = arpes_par['nkx'], arpes_par['nky']
	kcart = kgrid.GenKgrid2D_cart(kxr, kyr, nk1, nk2, kdim=2)
	kpts = kgrid.FracToCart2D(kcart, exc.brec[:,0], exc.brec[:,1])

	ws = np.array([1.715])

	for iexc in range(1,exc.nst):
		exc_en = exc.eps[iexc]
		exc_dens = np.abs(exc.excwf[iexc,:])**2
		# pexc = pexcs[iexc] - np.array([0.0, 0.5])
		pexc = pexcs[iexc]
		spect = arpes.GetIntensity(kpts, ws/cst.Ry, exc_en, exc_dens, pexc, iv, eta/cst.Ry)

		# if np.abs(pexc[0]) > 0:
		# 	pexc = pexcs[iexc] - np.array([0.5, 0.0])
		# 	spect += arpes.GetIntensity(kpts, ws/cst.Ry, exc_en, exc_dens, pexc, iv, eta/cst.Ry)
		# 	pexc = pexcs[iexc] - np.array([0.0, 0.5])
		# 	spect += arpes.GetIntensity(kpts, ws/cst.Ry, exc_en, exc_dens, pexc, iv, eta/cst.Ry)

		PlotIntensity(spect,kxr,kyr,nk1,nk2)	
#===========================================================================
if __name__ == '__main__':
	main(sys.argv[1:])